In [0]:
pip install jsonlines

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import jsonlines
import json
from glob import glob
l=[]
with open('train.jsonl', "r") as f:

    for line in f.readlines():
        l.append(json.loads(line))
        
train_set= l[0:int(len(l)*0.8)]

test_set = l[int(len(l)*0.8)+1:]

train_loader=[]

for i in train_set:

    for j in range(4):
        
        train_loader.append([i["parent"]+" <SEP> " +i["response"] , i["child_comment_" + str(j)]])
#taking transpose of the train_loader
train_loader= list(map(list, zip(*train_loader)))


test_loader_contexts =[]
test_loader_responses =[]


for i in test_set:
    
        test_loader_contexts.append([str(i["parent"]+" <SEP> " +i["response"]), str(i["parent"]+" <SEP> " +i["response"]),str(i["parent"]+" <SEP> " +i["response"]),str(i["parent"]+" <SEP> " +i["response"])] )
        test_loader_responses.append([i["child_comment_" + str(0)],i["child_comment_" + str(1)],i["child_comment_" + str(2)],i["child_comment_" + str(3)],i["label"]])
        
    

#test_loader = list(map(list, zip(*test_loader)))




In [0]:
import numpy as np
import scipy.sparse as sp
from sklearn.feature_extraction.text import (HashingVectorizer,
                                             TfidfTransformer,
                                             _document_frequency)
from sklearn.utils.testing import ignore_warnings
import method


class TfIdfMethod(method.BaselineMethod):

    """TF-IDF baseline.
    This hashes words to sparse IDs, and then computes tf-idf statistics for
    these hashed IDs. As a result, no words are considered out-of-vocabulary.
    """
    def train(self, contexts, responses):
        """Fit the tf-idf transform and compute idf statistics."""
        self._vectorizer = HashingVectorizer()
        self._tfidf_transform = TfidfTransformer()
        
        self._tfidf_transform.fit(
            self._vectorizer.transform(contexts + responses))

    def _vectorize(self, strings):
        """Vectorize the given strings."""
        tf_idf_vectors = self._tfidf_transform.transform(
            self._vectorizer.transform(strings))
        return sp.csr_matrix(
            tf_idf_vectors, dtype=np.float64, copy=True)

    def rank_responses(self, contexts, responses):
        """Rank the responses for each context."""
        contexts_matrix = self._vectorize(contexts)
        responses_matrix = self._vectorize(responses)
        similarities = contexts_matrix.dot(responses_matrix.T).toarray()
        return np.argmax(similarities, axis=1)







In [0]:
def train(model):
  model= TfIdfMethod() 
  model.train(train_loader[0],train_loader[1])
  return model


def evaluate(model):
    total =0
    correct =0
    for i in range(len(test_loader_contexts)):
        a = test_loader_contexts[i]
        b = test_loader_responses[i]
        predicted_label = model.rank_responses([a[0],a[1],a[2],a[3]],[b[0],b[1],b[2],b[3]])[0]
        #check if label matches
        #print(b[4])
        #print(predicted_label)

        #print("A")
        if int(predicted_label)==int(b[4]):
            correct+=1
        total+=1
        
        #print(predicted_label)
    return correct/total
model = None
model = train(model)
evaluate(model)

0.3721028234302571